# Imports & Global Variables

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from openpyxl.workbook import Workbook
import pandas_profiling

from pathlib import Path
from datetime import date

%load_ext lab_black

In [2]:
TODAY = pd.Timestamp.today()
DATA_DIR = Path.cwd() / "raw_data"

print(f"[Path.home()] {Path.home()}")
print(f"[Path.cwd]    {Path.cwd()}")
print(f"[DATA_DIR]    {DATA_DIR}")
print(f"[TODAY]       {TODAY}")

[Path.home()] C:\Users\stc
[Path.cwd]    C:\Users\stc\Documents\GitHub\sanitas_aufsichtsdaten
[DATA_DIR]    C:\Users\stc\Documents\GitHub\sanitas_aufsichtsdaten\raw_data
[TODAY]       2021-06-08 18:09:16.969214


# Load raw data

In [141]:
bag_df = pd.DataFrame()

for year in range(2010, 2020):
    bag_year_df = pd.read_excel(
        DATA_DIR / "aufsichtsdaten-okp-1996-2019.xlsx",
        sheet_name=f"{year}",
        usecols="A:AF",
        skiprows=9,
        nrows=64,
    )
    bag_year_df["year"] = year
    bag_df = pd.concat([bag_year_df, bag_df])
bag_df = bag_df[~bag_df[0].isna()]

bag_df = (
    bag_df.sort_values([0, "year"], ascending=[True, False])
    .reset_index()
    .drop(columns="index")
)
rename_columns = list(bag_df.columns.copy())
rename_columns[0] = "bag_nr"
rename_columns[1] = "versicherung"
rename_columns[2] = "bestand"
bag_df.columns = rename_columns

newest_names_df = (
    bag_df.loc[bag_df.year == 2019, ["bag_nr", "versicherung"]].drop_duplicates().copy()
)

bag_df = pd.merge(
    bag_df.drop(columns=["versicherung"]),
    newest_names_df,
    left_on="bag_nr",
    right_on="bag_nr",
    how="inner",
)

# Data Preparation

Analysiere die Aufsichtsdaten OKP (Obligatorische Krankenpflegeversicherung) der 10 grössten OKP Versicherer inkl. der zwei Sanitas Carrier (Sanitas, Compact) für die Jahre 2015 – 2019 und veranschauliche die wichtigsten Erkenntnisse grafisch. Erläutere deine Vorgehensweise.

In [146]:
top10 = (
    bag_df[bag_df.year == 2019].sort_values("bestand", ascending=False).head(10).bag_nr
)
bag_nr_sanitas_compact = pd.Series([1575, 1509])

relevant_bag_nr = top10.append(bag_nr_sanitas_compact)

In [147]:
selection_df = bag_df[bag_df.bag_nr.isin(relevant_bag_nr)]

In [148]:
selection_df

,bag_nr,bestand,3B,5B,13B,12B,4B,6B,7,8B,...,11C,11,6C,year,Unnamed: 15,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,versicherung
0,8.0,861448.88200,3698.301661,3871.284181,561.347076,4432.631257,-371.352974,138.296179,239.001422,810.785195,...,NaN,125.530412,0.033982,2019,NaN,NaN,NaN,NaN,NaN,CSS Kranken-Versicherung AG
1,8.0,842321.26990,3669.954364,3793.792107,564.190840,4357.982947,-369.715929,117.839602,93.772483,931.651649,...,NaN,105.803099,0.029171,2018,NaN,NaN,NaN,NaN,NaN,CSS Kranken-Versicherung AG
2,8.0,811078.49050,3638.879143,3749.601106,546.461981,4296.063087,-440.806741,136.410577,131.157845,955.641173,...,NaN,124.201846,0.033437,2017,NaN,NaN,NaN,NaN,NaN,CSS Kranken-Versicherung AG
3,8.0,807333.20180,3558.237314,3714.690007,540.421301,4255.111308,-398.179009,133.992297,118.165921,946.573234,...,NaN,121.904726,0.033867,2016,NaN,NaN,NaN,NaN,NaN,CSS Kranken-Versicherung AG
4,8.0,837200.75910,3386.825826,3493.648310,522.311146,4015.959456,-297.440016,145.255353,2.275549,880.792321,...,NaN,134.871043,0.039426,2015,NaN,NaN,NaN,NaN,NaN,CSS Kranken-Versicherung AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1575.0,29348.24255,2678.689195,1192.910797,359.488847,1552.399644,1182.161090,180.603446,18.325703,359.942132,...,NaN,180.603446,0.120682,2015,NaN,NaN,NaN,NaN,NaN,Compact Grundversicherungen AG
487,1575.0,26174.80000,2678.698913,1141.341035,345.374566,1486.715601,1141.273091,182.787855,269.277597,347.839105,...,NaN,136.512260,0.118892,2014,NaN,NaN,NaN,NaN,NaN,Compact Grundversicherungen AG
488,1575.0,28247.97000,2673.948239,1227.687212,343.034303,1570.721515,1203.005597,153.102177,104.599532,394.236719,...,NaN,109.316385,0.104084,2013,NaN,NaN,NaN,NaN,NaN,Compact Grundversicherungen AG
489,1575.0,35082.10000,2608.528139,1196.981158,322.313557,1519.294715,1126.591396,168.746261,101.638428,463.720359,...,NaN,131.638378,0.113869,2012,NaN,NaN,NaN,NaN,NaN,Compact Grundversicherungen AG


***

__Kommentare__
1. Beim Datensatz handelt es sich ausschliesslich um das OKP-Geschäft.
2. Im OKP-Geschäft darf der Versicherer gesetzlich keinen Gewinn machen.
3. Profit wir lediglich VVG-Bereich gemacht.
4. Jeder OKP-Versicherte ist eine Chance für ein Cross-Sale. -> Der Bestand ist daher interessant
5. Jedoch sind nur "Gesunde" OKP-Versicherte von potentiellem Interesse. -> Risikoausgleich
6. Als Versicherer möchten wir den Aufwand minimieren, um mehr Kapazität für den VVG-Bereich zu schaffen. -> Verwaltungsaufwand 6B

__Welche KPIs betrachten wir?__
1. Risikoausgleich
2. Bestand
3. 

***

In [140]:
106 / 10

10.6